In [1]:
%pip install --upgrade pip
%pip install --upgrade transformers datasets[audio] accelerate

Note: you may need to restart the kernel to use updated packages.
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/10.5 MB ? eta -:--:--
   ----- ---------------------------------- 1.3/10.5 MB 7.4 MB/s eta 0:00:02
   ------------- -------------------------- 3.4/10.5 MB 8.4 MB/s eta 0:00:01
   --------------------- ------------------ 5.5/10.5 MB 8.8 MB/s eta 0:00:01
   ----------------------------- ---------- 7.6/10.5 MB 9.4 MB/s eta 0:00:01
   ----------------------------------- ---- 9.2/10.5 MB 9.0 MB/s eta 0:00:01
   ---------------------------------------- 10.5/10.5 MB 8.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ------------------------------ --------- 1.8/2.4 MB 10.1 MB/s eta 0:00:01
   ---------------------------------------- 2.4/2.4 MB 7.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0

In [6]:
import os
os.environ["PATH"] += os.pathsep + r"C:\Users\kyoohyun\Desktop\tool\ffmpeg-2025-06-02-git-688f3944ce-full_build\bin"

In [7]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
# from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
    return_timestamps=True,
    chunk_length_s=10,
    stride_length_s=2
)

# dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
# sample = dataset[0]["audio"]
sample = './audio/lsy_audio_2023_58s.mp3'

result = pipe(sample)
print(result)


Device set to use cpu
c:\Users\kyoohyun\Desktop\llm-practice\venv\Lib\site-packages\transformers\models\whisper\generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


{'text': ' 안녕하세요. 이 강의는 GPT-API로 챗봇 만들기 라는 내용을 다루는 강의입니다. GPT-API에 대해서 생소하신 분들도 있을텐데 우리가 잘 알고 있는 ChatGPT, ChatGPT 기능을 이용해서 우리가 원하는 프로그램을 어떻게 만드는지에 대해서 이야기할 거예요. 그래서 이런 강의들이 사실 많이 있습니다. 그래서 여러가지들이 있는데 이 강의 특징이라고 한다면 GPT로 명확한 미션을 달성하는 챕터 프로그램을 만드는게 사실 쉽지는 않은데 이걸 어떻게 해서 구현을 하는지 그리고 그게 왜 필요한지에 대해서 좀 이야기를 할 거고요. 그 예제로 예제는 여러가지가 될 수 있는데 예제로 하는 것은 음악 플레이리스트 동영상을 자동으로 대화를 통해서 생성하는 프로그램을 만드는 것을 다루려고 합니다. 프로그램이 실행되는 모습을 한번 보여드릴게요. 우리가 만들 프로그램은 이런 식으로 이제 나타나게 되고', 'chunks': [{'timestamp': (0.0, 6.3), 'text': ' 안녕하세요. 이 강의는 GPT-API로 챗봇 만들기 라는 내용을 다루는 강의입니다.'}, {'timestamp': (7.18, 10.0), 'text': ' GPT-API에 대해서 생소하신 분들도 있을텐데'}, {'timestamp': (11.0, 17.0), 'text': ' 우리가 잘 알고 있는 ChatGPT, ChatGPT 기능을 이용해서'}, {'timestamp': (17.0, 20.0), 'text': ' 우리가 원하는 프로그램을 어떻게 만드는지에 대해서'}, {'timestamp': (20.0, 22.0), 'text': ' 이야기할 거예요.'}, {'timestamp': (22.0, 24.0), 'text': ' 그래서 이런 강의들이 사실 많이 있습니다.'}, {'timestamp': (24.0, 27.48), 'text': ' 그래서 여러가지들이 있는데 이 강의 특징이라고 한다면'}, {'timestamp': (27.48, 29.58), 'text': ' G

In [8]:
start_end_text = []

for chunk in result["chunks"]:
    start = chunk["timestamp"][0]
    end = chunk["timestamp"][1]
    text = chunk["text"]
    start_end_text.append([start,end,text])

import pandas as pd
df = pd.DataFrame(start_end_text, columns=["start", "end", "text"])
df.to_csv("lsy_audio_2023_58.csv", index=False, sep="|")
display(df)




,start,end,text
0,0.00,6.30,안녕하세요. 이 강의는 GPT-API로 챗봇 만들기 라는 내용을 다루는 강의입니다.
1,7.18,10.00,GPT-API에 대해서 생소하신 분들도 있을텐데
2,11.00,17.00,"우리가 잘 알고 있는 ChatGPT, ChatGPT 기능을 이용해서"
3,17.00,20.00,우리가 원하는 프로그램을 어떻게 만드는지에 대해서
4,20.00,22.00,이야기할 거예요.
5,22.00,24.00,그래서 이런 강의들이 사실 많이 있습니다.
6,24.00,27.48,그래서 여러가지들이 있는데 이 강의 특징이라고 한다면
7,27.48,29.58,GPT로 명확한 미션을 달성하는
8,29.58,31.66,챕터 프로그램을 만드는게 사실
9,31.66,34.32,쉽지는 않은데 이걸 어떻게 해서
